# CP423: Assignment 1

Group Number: 5

Group Members: Abigail Lee (200469770), Kayleigh Habib (200370580) and Myisha Chaudhry (200591740)

In [1]:
# MIGHT NEED TO RUN THIS FOR THE NLTK
#pip install certifi

In [2]:
import certifi
import ssl
import os

os.environ["SSL_CERT_FILE"] = certifi.where()

In [3]:
# inputs
import os
import re

import nltk
nltk.download('punkt')
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kayleighhabib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kayleighhabib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Question 1 : Text Preprocessing
a. Convert all text to lowercase

b. Tokenize the text using NLTK

c. Remove stop words using NLTK

d. Exclude special characters except alphanumeric characters

e. Eliminate singly occurring characters

f. Create a set of all the words

In [4]:
def text_preprocessing_str(query):

    # a. converting to lowercase
    lines = [x.lower() for x in query]

    # join all the lines into one string
    data = "".join(lines)

    # b. using the tokenization package to tokenize the words
    tokenization = word_tokenize(data)

    # remove duplicates - less words to do the next steps with
    newtoken = list(dict.fromkeys(tokenization))

    # c. remove any stop words from the NLTK package
    stop_words_lib = set(stopwords.words("english")) # choosing the english stopwords from the package

    # goes through each word, and checks if it is in the library - if it is not then we keep the word, if it is we don't take it
    stop_words = []
    for word in newtoken:
            if word not in stop_words_lib:
                 stop_words.append(word)

    # print(stop_words)

    # d. Remove special characters
    pattern = "[^a-z0-9]" # pattern NOT with any letter between a-z or any number including 0-9 (could also do A-Z but since we already made everything lower don't need to add that)
    alphanumerictxt = []
    # goes through each index in the list and if it matches the pattern it will put an empty string otherwise will preserve the value
    # strips the empty string afterwards
    for i in stop_words:
        alphanumerictxt.append(re.sub(pattern," ", i).strip())

    # e. eliminating any characters of length 1
    updated_list = []

    # goes through each index in the list and if the length is more than 1 will append to new list
    for i in alphanumerictxt:
        if len(i)>1:
            updated_list.append(i)

    # after all the other steps are completed check again to make sure no duplicated values
    final_list = list(dict.fromkeys(updated_list))


    return final_list

In [5]:
def text_preprocessing(file):

    # reading the file line by line
    lines = file.readlines()

    # a. converting to lowercase
    lines = [x.lower() for x in lines]

    # join all the lines into one string
    data = "".join(lines)

    final_list = text_preprocessing_str(data)

    #f
    return final_list


In [6]:
# import data files
def readfile_fun():
  # takes the names of the files in the folder and creates a list
  file_names = os.listdir("./data")
  # file_names

  # file id (key)
  counter = 1

  #dictionary1 - translates the ID which belong to the file
  dict_words = {}
  # dictionaty2 - file dictionary (id to file name)
  dict_files = {}

  # goes through each file in the list of file names
  for i in file_names:

      # if counter <= 2: #for checking purposes - can remove before submitting

        # gets the working directory (this may be different for everyone)
        file_path = os.getcwd() + "/data/" + i
        # opens the file in reading mode
        file = open(file_path, "r", encoding="utf-8", errors = "ignore" ) # added the encoding and errors as some files had different encoding detectors (note: with new line can add new param --> newline = '/r/n'))
        # calls the function written for question 1 of the assignment
        words = text_preprocessing(file)

        # updates the dictionaries
        dict_words[counter] = words # list of words in this file
        dict_files[counter] = i # list of all the file names

      # increment the counter
        counter+=1

      # closes the file
        file.close()
  
  print(dict_files)
  print(dict_words)

  return dict_words, dict_files

## Question 2 : Inverted Index Implementation
Implement an inverted index data structure for the preprocessed dataset


In [7]:
class HashInvertedIndex:

    #initialize hash table
    def __init__(self, capacity=100):
        self.capacity = capacity
        self.table = [[] for _ in range(capacity)]
        self.count = 0

    #finding the corresponding value for the key
    def _index(self, word):
        index = hash(word) % self.capacity
        return index

    #add key-value pair to the hash table
    def insert(self, key, value):

        #compute the hash index for the word
        hash_index = self._index(key)
        for i, kv in enumerate(self.table[hash_index]):
            k, v = kv
            if k == key:
                #add the value to the existing set of values
                v.add(value)
                #add the entry to the table
                self.table[hash_index][i] = (k,v)
                return
        self.table[hash_index].append((key, {value}))

    #retrieve the set of values for a given key, if not found, returns an empty set
    def retrieve(self, key):
        #compute the hash index for a given key
        hash_index = self._index(key)
        for k, v in self.table[hash_index]:
            #if the key if found, return the set
            if k == key:
                return v
        return set()

    #for testing purposes - shows us the string representation of the hash tables
    def __rep__(self):
        return str(self.table)


In [8]:
#build the inverted index
def create_inverted_index():

    inverted_index = HashInvertedIndex()
    
    # calls the function above
    words, files = readfile_fun()
    
    for key, value in words.items():
        for word in value:
            inverted_index.insert(word, key) #inserting counter instead of doc name (change later?)

    return inverted_index, files, words


In [9]:
# calls the function
inverted_index, file_list, word_list = create_inverted_index()

# prints out the index and the list of file names
# index to first list is hash to the word
for i in range(len(inverted_index.table)):
    #for j in i:
    # prints the hash buckets
    print(inverted_index.table[i])
    # prints the length of each bucket (right now around 400-500 - evenly distributed) - if we want smaller amounts in each bucket set bucket_size = 200
    #print(len(inverted_index.table[i]))

#print(file_list)


[('hungrier', {143, 1, 202, 47}), ('wonderful', {1, 130, 132, 137, 139, 140, 15, 145, 18, 146, 147, 148, 153, 27, 29, 160, 161, 162, 163, 37, 40, 168, 172, 185, 189, 62, 194, 195, 81, 209, 83, 89, 91, 223, 98, 103, 105, 236, 110, 111, 241, 242, 243, 116, 245, 122, 125}), ('draped', {2, 6, 15, 111, 144, 209, 51, 180, 212, 242, 216, 186, 29, 62}), ('squdron', {3}), ('squad', {98, 3, 4, 101, 200, 45, 208, 241, 19, 179, 21, 191}), ('proper', {3, 133, 142, 15, 144, 18, 29, 39, 168, 42, 172, 47, 179, 186, 62, 190, 194, 76, 209, 92, 223, 241, 242, 245, 120, 122}), ('45', {65, 98, 3, 195, 103, 105, 236, 45, 111, 112, 21, 125, 23, 120, 29, 62}), ('sped', {130, 3, 40, 168, 170, 45, 179, 215, 216, 186, 191}), ('knifed', {218, 4, 14}), ('best', {4, 5, 6, 7, 8, 9, 15, 16, 19, 20, 21, 23, 25, 27, 29, 34, 36, 39, 40, 45, 47, 49, 52, 53, 54, 55, 56, 62, 72, 73, 74, 76, 81, 87, 88, 91, 98, 102, 104, 105, 111, 117, 119, 120, 122, 124, 125, 126, 131, 133, 138, 142, 143, 145, 146, 147, 149, 150, 162, 164,

## Question 3: Query Support
Support the following queries, where x and y would be taken as input from the user

1. x OR y

2. x AND y

3. x AND NOT y

4. x OR NOT y


In [26]:
# def query(sentence, operation, InvertedIndex, file):
#   matches = 0
#   comparisons = 0
  
#   index = 0

#   while index < len(operation):
#     print(sentence)
#     print("o: ", operation)
#     #OR
#     if operation[index] == "OR":
#     #if "OR" in operation:
#       #use a loop to find the index in the operation array where OR exists
#       #for item in operation:
#         # if item == "OR":
#         #   break

#       #based on the index found, find the words in the sentence array that the operation is performed on
#       word1 = sentence[index]
#       word2 = sentence[index + 1]
#       print("OR: ", word1)
#       print("OR: ", word2)

#       #find the inverted indexes for the two words
#       list1 = InvertedIndex.retrieve(word1)
#       list2 = InvertedIndex.retrieve(word2)

#       #using the two inverted indexes to find the union of both (documents where either word exists)
#       list_docs = list1.union(list2)

#       #add total documents to comparisons variable
#       matches += len(list_docs)
#       # looking at the minimum of the smallest list as UNION will put every value from the min list into the max list removing duplicates
#       comparisons += min(len(list1),len(list2))

#     #AND
#     elif operation[index] == "AND":

#     # #use a loop to find the index in the operation array where AND exists
#     #   for item in operation:
#     #     if item == "AND":
#     #       break
#     #     index += 1

#     #based on the index found, find the words in the sentence array that the operation is performed on
#       word1 = sentence[index]
#       word2 = sentence[index + 1]

#     #find the inverted indexes for the two words
#       list1 = InvertedIndex.retrieve(word1)
#       list2 = InvertedIndex.retrieve(word2)

#     #using the two inverted indexes to find the intersection of both (documents where both words exists)
#       list_docs = list1.intersection(list2)

#     #add total documents to comparisons variable
#       matches += len(list_docs)

#       # traverse through smaller list to get same value
#       comparisons += min(len(list1),len(list2))

#     #AND NOT
#     #lion minus stood (lion - intersection)
#     elif operation[index] == "AND NOT":

#     # #use a loop to find the index in the operation array where AND NOT exists
#     #   for item in operation:
#     #     if item == "AND NOT":
#     #       break
#     #     index += 1

#     #based on the index found, find the words in the sentence array that the operation is performed on
#       word1 = sentence[index]
#       word2 = sentence[index + 1]
#       print("AND NOT: ", word1)
#       print("AND NOT: ", word2)

#     #find the inverted indexes for the two words
#       list1 = InvertedIndex.retrieve(word1)
#       list2 = InvertedIndex.retrieve(word2)
#       # print(list1)
#       # print("list2:", list2)

#     #using the two inverted indexes to find where the first one exists but the second does not
#       list_docs = list1 - (list1.intersection(list2))

#     #add total documents to comparisons variable
#       matches += len(list_docs)
      
#       # to get the intersection
#       comparisons += min(len(list1),len(list2))
#       comparisons += len(list1.intersection(list2))

#     #OR NOT
#     elif operation[index] == "OR NOT":

#     # #use a loop to find the index in the operation array where OR NOT exists
#     #   for item in operation:
#     #     if item == "OR NOT":
#     #       break
#     #     index += 1

#     #based on the index found, find the words in the sentence array that the operation is performed on
#       word1 = sentence[index]
#       word2 = sentence[index + 1]
#       print("OR NOT: ", word1)
#       print("OR NOT: ", word2)

#     #find the inverted indexes for the two words
#       list1 = InvertedIndex.retrieve(word1)
#       list2 = InvertedIndex.retrieve(word2)
#       # print("OR NOT: ", list1)
#       # print("list2:", list2)

#     #using the two inverted indexes to find where the first word exists or the second word does not
#       temp = list2 - (list2.intersection(list1))
#       #temp = list2.union(list1.intersection(list2))
#       list_docs = set(file.values()) - temp

#       matches += len(list_docs)

#       comparisons += min(len(list1),len(list2))
#       comparisons += len(list1.intersection(list2))
#       comparisons += len(temp)

#     else: 
#       print("else")
#     index+= 1
    
#   return matches, comparisons

In [44]:
def query2(sentence, operation, InvertedIndex, file):
  comparisons = 0
  index = 0
  word = sentence[index]
  final_list = InvertedIndex.retrieve(word)

  while index < len(operation):
    #OR
    if operation[index] == "OR":
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # looking at the minimum of the smallest list as UNION will put every value from the min list into the max list removing duplicates
      comparisons += min(len(final_list),len(list1))

      final_list = final_list.union(list1)

    #AND
    elif operation[index] == "AND":
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # looking at the minimum of the smallest list as UNION will put every value from the min list into the max list removing duplicates
      comparisons += min(len(final_list),len(list1))

      final_list = final_list.intersection(list1)

    elif operation[index] == "AND NOT":
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # to get the intersection
      comparisons += min(len(final_list),len(list1))
      comparisons += len(final_list.intersection(list1))

      final_list = final_list - (final_list.intersection(list1))

    elif operation[index] == "OR NOT":
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)


      #using the two inverted indexes to find where the first word exists or the second word does not
      temp = list1 - (list1.intersection(final_list))
      
      comparisons += min(len(final_list),len(list1))
      comparisons += len(final_list.intersection(list1))
      comparisons += len(temp)

      final_list = set(file.values()) - temp

    index+=1

  return final_list, comparisons
      


## Question 4: System Evaluation
Evaluate your system against the set of provided queries.

1. Query #1

- Input Sentence: "lion stood thoughtfully for a moment"

- Input operation sequence: [OR, OR, OR]

- Expected preprocessed query: "lion OR stood OR thoughtfully OR moment"

2. Query #2

- Input Sentence: "telephone, paved, roads"

- Input operation sequence: [OR NOT, AND NOT]

- Expected preprocessed query: "telephone OR NOT paved AND NOT roads


Output should include:
- The number of documents retrieved

- The minimum number of total comparisons made (only for merging the algorithm)

- The list of retrieved document names

In [46]:
text = input("Enter your string: ")
sentence_list = text_preprocessing_str(text)

operations = input("Enter the operations (in CAPS, seperated with commas, NO SPACES between the operations): ")
operations_list = operations.split(",")
print("Sentence: " ,sentence_list)
print("Operations list: ", operations_list)
match_list, comp = query2(sentence_list, operations_list, inverted_index,file_list)


print("Number of Matched documents: " , len(match_list))
print("Minimum number of comparisons required: ", comp)
print("List of retreived document names: ")
for match in match_list:
     print(file_list[match])

Sentence:  ['lion', 'stood', 'thoughtfully', 'moment']
Operations list:  ['AND', 'AND', 'AND']
Number of Matched documents:  1
Minimum number of comparisons required:  23
List of retreived document names: 
lionwar.txt
